! jt -r

In [1]:
import pandas as pd
import src.daily_data as dt
import src.auxiliary_functions as aux
import src.condition as con
from datetime import time, datetime
import src.info as info
import src.in_out as io

from src.density import Density

1. **data** klasöründeki tüm dosya yolları alınıyor

In [2]:
path_list = io.get_path('C:\\Users\\ugur.eren\\Python Codes\\cefis2\\data\\')

2. *Her bir path'de bulunan veriler **dataframe** olarak alınıyor. Her bir df, **split_df** metodu ile saatlik olarak **23** ayrı df'e bölünüyor. Bölünmüş her bir df'in **mid_price**'ı hesaplanıp yeni bir sütun olarak **df**'e ekleniyor.* **get_info** metodu her bir path için bu verileri kullanarak bir **info** örneği oluşturup döndürür. Her bir **info** örneği, aşağıdaki özelliklere sahiptir:<br>
    * Okunan dosyanın tarihi
    * Ürünün adı
    * Okunan dosyaya ait yol
    * Tüm verileri içeren dataframe
    * Saatlik olarak bölünmüş dataframe listesi


In [3]:
infolist = [info.get_info(path) for path in path_list]

3. Burada, oluşturulan infolar arasında bir **eşleştirme** yapılmaktadır. Çünkü her bir **spread** hesabının hangi veriler kullanılarak yapılacağı bilinmelidir. Eşleşen infolar, tuple çiftleri olarak tutulmakta.
  Elimizde 2018:09:06 tarihinin 6A,6B,6C ürünlerine ait verileri olsun. Bu durumda 6A-6B, 6A-6C, 6B-6C arasında 3 adet spread hesaplanacaktır. Hesaplama öncesi bu eşleştirme, **daily_data** modülünün **to_match_days** metoduyla yapılmaktadır.

In [4]:
paired_days = dt.to_match_days(infolist)

4.  **daily_datas**: GünlükVeri örneklerinin tutulduğu liste <br />
**dt.to_match_hour** metodu ile her bir eşleşmiş çiftin mid_price'ları saatlik olarak alınıp birleştiriliyor. <br />
**gunluk_veri**: **tarih**, her bir saate ait **change**,**spread**,**amplitude**,**duration**,**urun adları**...vs içeriyor <br />
**paired_hours:** 23 adet df içerir. her bir df mid_price çifti içerir

In [5]:
daily_data = []

In [6]:
for pair in paired_days:
    product = pair[0].product + "_" + pair[1].product   # urun adi
    paired_hours=dt.to_match_hour(pair[0],pair[1])
    gunluk_veri = dt.DailyData(mid_price_list=paired_hours,date=pair[0].date, name= product)
    gunluk_veri.spread_hesapla()
    daily_data.append(gunluk_veri)

5. Her bir **DailyData** örneği ürünlere göre ayrı ayrı gruplandırılıyor. <br>
    (['**6AU8_6BU8**': [gunluk veriler...], '**6AU8_6CU8**':[gunluk veriler...], '**6BU8_6CU8**':[gunluk veriler...]])

In [7]:
products={}

In [8]:
for gunluk in daily_data:
    if gunluk.name in products:
        products[gunluk.name].append(gunluk)
    else:
        products[gunluk.name] = [gunluk]

**combined_days :**  key-value olarak ürün adları ve verileri içerir <br />
**combined_days_detail :** key-value olarak ürün adları ve verilere ait istatistiksel bilgileri içerir <br />
**combined_days_con :** key-value olarak ürün adları ve median koşuluna göre yeniden düzenlenmiş verileri içerir <br />
* 6BU8_6CU8'e ait verilere erişmek için:<br />
 ```python
combined_days_detail['6BU8_6CU8']
```
* 6BU8_6CU8'e ait 18:00:00-19:00:00 saatleri arası verilere erişmek için: <br />
 ```python
combined_days['6BU8_6CU8'].loc[time(18)]
```

In [9]:
combined_days = {}
combined_days_detail = {}
combined_days_con = {}
combined_days_con_detail = {}

In [10]:
stop=False
i=0
while products:
    urun = products.popitem()
    df = dt.get_all_data(urun[1]).dropna()
    con_df = con.scan(df)
    combined_days[urun[0]] = df
    combined_days_con[urun[0]] = con_df
#    combined_days_detail[urun[0]] = aux.get_detail(df)
#     combined_days_con_detail[urun[0]] = aux.get_detail(con_df)

Hesaplanmış ürün adları ve saat bilgisi alınıyor

In [11]:
all_products = list(combined_days.keys())

In [12]:
hour_series = pd.date_range('2018-01-01-18', periods=23, freq='H').time

all_density:
```json
{"6BU8_6CU8": {"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 "6AU8_6CU8":{"18:00:00": "Density Object", "19:00:00": "Density Object"...},
 ....}
```

In [36]:
all_density= {}

In [37]:
for product in all_products:
    densities = {}
    for hour in hour_series:
        df = combined_days[product].loc[hour]
        densities[hour]= Density(df)
    all_density[product] = densities

--------------------------------

# Example

In [46]:
df = data_of_the_eighteenth_clock.density_data

In [47]:
df

,duration,amplitude,density
0,1.0,-10.000,0.004016
1,1.0,-10.000,0.004016
2,1.0,-6.250,0.004016
3,1.0,-6.250,0.004016
4,1.0,-5.000,0.004016
5,1.0,-5.000,0.004016
6,1.0,-3.125,0.040161
7,1.0,-3.125,0.040161
8,1.0,1.875,0.004016
9,1.0,1.875,0.004016


In [93]:
df_group = df.groupby(['duration','amplitude']).sum()

In [101]:
df_piv = pd.pivot_table(df, values='density', index=['duration'],columns='amplitude')

In [116]:
df_piv

amplitude,-13.125000000003638,-13.125000000002718,-11.87500000001819,-11.875000000016733,-11.250000000025466,-11.25000000002374,-10.0,-9.999999999998899,-9.375000000007276,-9.375000000005906,...,9.375000000007276,9.999999999998899,10.0,10.000000000010001,10.000000000010914,13.124999999991616,13.124999999992724,20.624999999990788,20.624999999992724,31250.0
duration,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.004016,0.004016,NaN,NaN,...,NaN,0.008032,0.008032,NaN,NaN,NaN,NaN,NaN,NaN,0.004016
2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,NaN,NaN,NaN,NaN,0.004016,0.004016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.004016,0.004016,NaN,NaN,NaN,NaN,NaN
5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df_group.unstack().transpose()

duration               1.0       2.0       3.0       4.0       5.0    \
        amplitude                                                      
density -13.125          NaN       NaN       NaN       NaN       NaN   
        -13.125          NaN       NaN       NaN       NaN       NaN   
        -11.875          NaN       NaN       NaN       NaN       NaN   
        -11.875          NaN       NaN       NaN       NaN       NaN   
        -11.250          NaN       NaN  0.004016       NaN       NaN   
        -11.250          NaN       NaN  0.004016       NaN       NaN   
        -10.000     0.004016       NaN       NaN       NaN       NaN   
        -10.000     0.004016       NaN       NaN       NaN       NaN   
        -9.375           NaN       NaN       NaN       NaN       NaN   
        -9.375           NaN       NaN       NaN       NaN       NaN   
        -8.125           NaN  0.004016       NaN       NaN       NaN   
        -8.125           NaN  0.004016       NaN       NaN       NaN   
        -8.125           NaN       NaN       NaN       NaN       NaN   
        -8.125           NaN       NaN       NaN       NaN       NaN   
        -8.125           NaN  0.004016       NaN       NaN       NaN   
        -8.125           NaN  0.004016       NaN       NaN       NaN   
        -6.250      0.004016       NaN       NaN       NaN       NaN   
        -6.250           NaN       NaN       NaN       NaN       NaN   
        -6.250      0.004016       NaN       NaN       NaN       NaN   
        -6.250           NaN  0.004016       NaN       NaN       NaN   
        -6.250           NaN  0.004016       NaN       NaN       NaN   
        -5.000           NaN       NaN       NaN       NaN       NaN   
        -5.000           NaN       NaN       NaN       NaN       NaN   
        -5.000      0.004016       NaN       NaN       NaN       NaN   
        -5.000      0.004016       NaN       NaN       NaN       NaN   
        -3.125           NaN       NaN       NaN       NaN       NaN   
        -3.125           NaN       NaN       NaN       NaN       NaN   
        -3.125      0.040161  0.008032  0.008032       NaN  0.008032   
        -3.125      0.040161  0.008032  0.008032       NaN  0.008032   
         1.875      0.004016       NaN       NaN       NaN       NaN   
         1.875      0.004016       NaN       NaN       NaN       NaN   
         3.125      0.044177  0.012048  0.004016  0.020080  0.004016   
         3.125      0.044177  0.012048  0.004016  0.020080  0.004016   
         3.125           NaN  0.004016       NaN       NaN       NaN   
         3.125           NaN  0.004016       NaN       NaN       NaN   
         5.000      0.008032  0.004016  0.004016  0.004016       NaN   
         5.000      0.008032  0.004016  0.004016  0.004016       NaN   
         6.250           NaN  0.004016       NaN  0.004016       NaN   
         6.250           NaN  0.004016       NaN  0.004016       NaN   
         6.250           NaN       NaN  0.004016       NaN       NaN   
         6.250           NaN       NaN  0.004016       NaN       NaN   
         6.250      0.004016       NaN  0.004016       NaN       NaN   
         6.250           NaN       NaN       NaN       NaN       NaN   
         6.250      0.004016       NaN  0.004016       NaN       NaN   
         8.125           NaN  0.004016       NaN       NaN  0.004016   
         8.125           NaN  0.004016       NaN       NaN  0.004016   
         8.125           NaN       NaN       NaN       NaN       NaN   
         8.125           NaN       NaN       NaN       NaN       NaN   
         9.375           NaN       NaN       NaN       NaN       NaN   
         9.375           NaN       NaN       NaN       NaN       NaN   
         10.000     0.008032       NaN       NaN       NaN       NaN   
         10.000     0.008032       NaN       NaN       NaN       NaN   
         10.000          NaN       NaN       NaN  0.004016       NaN   
         10.000          NaN       NaN  

In [45]:
data_of_the_eighteenth_clock = all_density['6AU8_6BU8'][time(18)]

data_of_the_eighteenth_clock.data

data_of_the_eighteenth_clock.density_data

In [ ]:
data_of_the_eighteenth_clock.draw()